# Spotify Dataset - Audio Features Clustering

## 1. Load Data
Kita akan memuat dataset yang kita ambil dari [Kaggle](https://www.kaggle.com/datasets/vatsalmavani/spotify-dataset).  
Berikut langkah-langkah untuk memuat data:  
1. Klik link [ini](https://www.kaggle.com/datasets/vatsalmavani/spotify-dataset), anda akan menuju ke kaggle.com
2. Klik tombol download di pojok kiri atas
3. Anda mungkin harus login ke kaggle.com untuk mendownload dataset
4. Tunggu sampai file sudah terdownload (archive.zip)
5. Ekstrak data.csv di dalam archive.zip, masukkan ke folder yang sama dengan file notebook (.ipynb)
6. Muat data.csv menggunakan kode dibawah, pastikan path benar

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist


import plotly.express as px 

# muat dataset
data = pd.read_csv('data.csv')
data.head()

In [ ]:
# visualisasi data duration menggunakan scatter plot
fig = px.scatter(data, x='loudness', y='popularity', color='loudness', 
hover_data=['artists','name','loudness','popularity'])
fig.show()

In [ ]:
# visualisasi data loudness menggunakan histogram plotly
counts, bins = np.histogram(data.loudness, bins=range(-60, 5, 1))
bins = 0.5 * (bins[:-1] + bins[1:])

fig = px.bar(x=bins, y=counts, labels={'x':'loudness', 'y':'count'})
fig.show()

In [ ]:
# visualisasi data loudness menggunakan diagram box plotly
fig = px.box(data,x='loudness', color='loudness')
fig.show()

## 2. Data Cleaning/Preprocessing

Berikut beberapa langkah yang anda bisa lakukan untuk melakukan proses data cleaning
- Cek nilai kosong (null)
- cek nilai duplikat
- Cek outlier

Namun sebelum kita lakukan data cleaning, kita ubah urutan kolom terlebih dahulu agar lebih enak untuk dilihat

In [ ]:
# pindahkan kolom agar lebih mudah dibaca
data = data[['artists','name' , 'year', 'popularity', 'key','mode', 'explicit', 'duration_ms', 'acousticness','danceability','energy','instrumentalness','loudness', 'liveness', 'speechiness', 'tempo','valence']]
data.head()

### Cek Nilai Kosong

In [ ]:
#cek nilai kosong
data.isnull().sum()

Tidak ada nilai kosong di dataset. Dataset aman.

Namun, jika ada data yang kosong, maka ada 2 opsi untuk menanganinya :
- Isi dengan nilai yang sesuai
- Hapus entry atau row  

Mengisi data yang kosong terkadang sangat sulit karena sifat data yang tidak dapat diprediksi dan tidak berkorelasi dengan kolom/fitur lainnya. Oleh karena itu, jika ada data yang kosong, lebih baik dihapus satu row/entry.

### Cek nilai duplikat

In [ ]:
#cek duplikat nama lagu
data['name'].duplicated().sum()

Ternyata ada 37015 nilai duplikat di kolom nama. Cukup banyak bukan? mari kita telusuri lebih dalam dengan mensortir dataset berdasarkan nama.  

In [ ]:
data.sort_values('name', inplace=False).iloc[150:180]

Ternyata lagu dengan judul Round Midnight mempunyai banyak duplikat, namun artis dan tahun rilisnya berbeda-beda. Kemungkinan besar lagu tersebut di cover oleh penyanyi lain, jadi masih termasuk lagu yang valid.  
Lagu yang tidak valid adalah duplikat artis, nama lagu dan tahun rilis. Contohnya di index 95937, 95798, 128106, dan 128159. Mereka mempunyai nama lagu dan artis yang sama, namun fitur lainnya cukup identik.  
Jadi kita bisa hapus data yang mempunyai nama lagu, artis dan tahun yang sama.

In [ ]:

#hilangkan duplikat data
data.drop_duplicates(subset=['name', 'artists', 'year'], inplace=False)
data.drop_duplicates(subset=['name', 'artists'], inplace=False)
data.sort_values('name', inplace=False).iloc[150:180]


Kita bisa lihat hasil data cleaning kita, sekarang hanya ada satu lagu dari Miles Davis yang berjudul 'Round Midnight

### Check Outlier
Karena dataset kita adalah musik, maka data nya cenderung sangat bervariasi, namun ada satu fitur yang dapat kita cek outliernya, yaitu durasi lagu.

In [ ]:
# check outlier
data['duration_ms'].describe()

Kita mendapatkan 2 nilai yang menarik dari analisis ini. Nilai minimum dan maximum.  
Nilai minimum mempunyai durasi 5108 miliseconds, atau cuma 5.1 detik.  
Nilai maximum mempunyai durasi 5403500 miliseconds, atau 5403 detik, atau 90 menit.  
Mari kita analisa kedua nilai ini, mulai dari nilai minimum.  
untuk mengecek nilai minimum, kita sortir dataset berdasarkan durasi dan urutkan dari kecil ke besar  

In [ ]:
data.sort_values('duration_ms', ascending=True).head()

Di urutan pertama dan kedua ada lagu bernama 'pause track' yang mempunyai durasi sangat pendek, jauh lebih pendek dari lagu lainnya. Selain itu, keempat data tersebut mempunyai nilai 0 di hampir semua fitur kecuali durasi dan loudness. Selanjutnya, kita akan hapus 4 data ini.

In [ ]:
# drop 4 data pal
data.sort_values('duration_ms', ascending=True).drop([22399, 22464, 142097, 78141], inplace=False).head()

## 2. Data Understanding - Data Description

Berikut adalah dataset yang digunakan untuk analisis ini. Dataset ini berisi data lagu-lagu yang ada di Spotify. Dataset ini terdiri dari 19 fitur, yaitu:
1. `name` : Nama lagu
2. `artists` : Nama artis pembuat lagu
3. `year` : Tahun rilis lagu
4. `popularity` : Tingkat popularitas lagu. Nilai popularitas adalah antara 0 dan 100, dengan 100 menjadi nilai maksimal.
5. `key` : adalah kunci lagu dari lagu tersebut. kunci lagu direpresentasikan menggunakan angka menurut Pitch Class Notation Standard. Misalnya, 0 = C, 1 = C♯/D♭, 2 = D, dan seterusnya. Jika tidak ada kunci, maka nilai -1.
6. `mode` : Mode dari lagu tersebut. 1 = Major, 0 = Minor.
7. `explicit` : Lagu tersebut mengandung konten eksplisit atau tidak. 1 = Ya, 0 = Tidak.
8. `duration_ms` : Durasi lagu dalam milidetik.
9. `acousticness` : Tingkat intensitas akustik di dalam lagu tersebut. 0.0 = Lagu tersebut tidak akustik, 1.0 = Lagu tersebut sangat akustik.
10. `danceability` : Seberapa cocok lagu tersebut untuk mengiringi tarian. Nilai diambil berdasarkan kombinasi dari beberapa elemen lain seperti tempo, stabilitas ritme, kekuatan ritme, dan reguleritas umum. 0.0 = Lagu tersebut tidak cocok untuk mengiringi tarian, 1.0 = Lagu tersebut sangat cocok untuk mengiringi tarian.
11. `energy` : Energi adalah ukuran dari 0.0 hingga 1.0 dan mewakili ukuran perseptual dari intensitas dan aktivitas. Secara umum, lagu-lagu yang energik terasa cepat, keras, dan berisik. Sebagai contoh, death metal memiliki energi yang tinggi, sementara prelude Bach memiliki skor rendah dalam skala tersebut. Fitur perseptual yang berkontribusi pada atribut ini termasuk rentang dinamis, kebisingan terdengar, timbre, tingkat onset, dan entropi umum.
12. `instrumentalness` : Memprediksi apakah sebuah lagu tidak mengandung vokal. Suara "ooh" dan "aah" diperlakukan sebagai instrumen dalam konteks ini. Lagu rap atau spoken word secara jelas dianggap "vokal". Semakin mendekati nilai instrumentalness ke 1.0, semakin besar kemungkinan lagu tersebut tidak mengandung konten vokal. Nilai di atas 0.5 dimaksudkan untuk mewakili lagu instrumental, namun tingkat kepercayaan lebih tinggi saat nilai mendekati 1.0.
13. `loudness` : Kekuatan suara keseluruhan dari sebuah lagu dalam desibel (dB). Nilai kekuatan suara dihitung rata-rata sepanjang seluruh lagu dan bermanfaat untuk membandingkan kekuatan suara relatif antara lagu-lagu. Kekuatan suara adalah kualitas dari suara yang merupakan korelasi psikologis utama dari kekuatan fisik (amplitudo). Nilai-nilai biasanya berkisar antara -60 hingga 0 dB.
14. `liveness` : Mendeteksi keberadaan penonton dalam rekaman. Nilai liveness yang lebih tinggi mengindikasikan probabilitas yang lebih besar bahwa lagu tersebut dipentaskan secara langsung. Nilai di atas 0.8 memberikan kemungkinan kuat bahwa lagu tersebut dinyanyikan secara langsung.
15. `speechiness` : Speechiness mendeteksi keberadaan kata-kata yang diucapkan dalam sebuah lagu. Semakin mirip rekaman tersebut dengan ucapan (misalnya, acara talk show, buku audio, puisi), semakin mendekati nilai atribut 1.0. Nilai di atas 0.66 menggambarkan lagu yang kemungkinan terdiri sepenuhnya dari kata-kata yang diucapkan. Nilai antara 0.33 dan 0.66 menggambarkan lagu yang mungkin mengandung baik musik maupun ucapan, baik dalam bagian-bagian atau lapisan, termasuk dalam kasus seperti musik rap. Nilai di bawah 0.33 kemungkinan besar mewakili musik dan lagu-lagu yang tidak mirip dengan ucapan.
16. `tempo` : The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.
17. `valence` : A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).

sumber : https://developer.spotify.com/documentation/web-api/reference/get-several-audio-features

Mari kita cek tipe data dari tiap fitur yang ada di dataset kita

In [ ]:
print(data.info())

data.
()

fitur artist dan name mempunyai data jenis object karena artist terdiri lebih dari satu penyanyi karena adanya lagu yang di fitur oleh penyanyi lain.  
jenis data int64 menandakan bahwa fitur mempunyai nilai integer simpel, tidak ada angka koma.
jenis data float64 menandakan bahwa fitur mempunyai nilai float yang mempunyai banyak angka dibelakang koma.

## 2. Data Understanding - Feature Corellation
Data understanding adalah proses untuk memahami data dan fitur fitur yang ada didalamnya. Salah satu cara untuk memahami sebuah data adalah melakukan eksperimen korelasi fitur. Korelasi fitur adalah sebuah metode untuk mengecek korelasi antara fitur dan dependent variable dengan cara memvisualisasikan hasil kalkulasi koefisien Pearson corellation dan informasi mutual antara fitur-fitur dan dependent variable. Fitur akan mempunyai nilai tertentu, tergantung dengan korelasinya terhadap dependent variable.  

Dependent variable yang kita pilih untuk analisis ini adalah popularitas.

In [ ]:
from yellowbrick.target import FeatureCorrelation

feature_names = ['acousticness', 'danceability', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'valence','duration_ms','explicit','key','mode','year']

X, y = data[feature_names], data['popularity']

# Create a list of the feature names
features = np.array(feature_names)

# Instantiate the visualizer
visualizer = FeatureCorrelation(labels=features)

plt.rcParams['figure.figsize']=(5,5)
visualizer.fit(X, y) # Fit the data to the visualizer
visualizer.show()

Berdasarkan hasil diatas, kita bisa menyimpulkan beberapa hal, bahwa lagu yang popular memiliki ciri-ciri sebagai berikut :  
- Tingkat akustik yang rendah
- Energi yang tinggi
- Rilis lebih baru
- Volume yang cukup keras

## 3.Clustering
Dari semua fitur-fitur tersebut kita akan melakukan clustering untuk mengelompokkan lagu-lagu tersebut. Clustering adalah proses pengklasifikasian dataset yang tidak mempunyai label, oleh karena itu, clustering termasuk kategori unsupervised machine learning.  

Dalam dataset ini, nama lagu akan kita clustering berdasarkan 18 fitur yang kita punyai.  

### K-Means

Algoritma clustering k-means adalah salah satu algoritma pembelajaran mesin yang populer dan digunakan untuk mengelompokkan titik-titik data ke dalam kelompok-kelompok berdasarkan kemiripannya. Algoritma ini termasuk dalam kategori algoritma pembelajaran tanpa pengawasan (unsupervised learning).

Berikut ini penjelasan langkah-langkah algoritma clustering k-means:

1. Inisialisasi: Secara acak, pilih K titik data awal sebagai pusat-pusat awal (centroid). Pusat-pusat ini mewakili titik-titik pusat kelompok awal.

2. Penugasan: Untuk setiap titik data dalam himpunan data, hitung jarak antara titik tersebut dengan setiap pusat. Berikan titik data ke kelompok yang memiliki pusat terdekat dengan titik tersebut. Langkah ini biasanya menggunakan jarak Euclidean, tetapi bisa juga menggunakan metrik jarak lainnya.

3. Pembaruan: Setelah mengalokasikan semua titik data ke dalam kelompok-kelompok, hitung kembali pusat kelompok (centroid) dari setiap kelompok. Pusat kelompok dihitung sebagai nilai rata-rata dari semua titik data yang termasuk dalam kelompok tersebut.

4. Iterasi: Ulangi langkah penugasan dan pembaruan secara berulang hingga konvergensi. Konvergensi terjadi ketika penugasan titik-titik data ke kelompok-kelompok tidak berubah lagi atau ketika mencapai batas iterasi maksimum yang telah ditentukan sebelumnya.

5. Hasil Akhir: Setelah algoritma konvergen, titik-titik data dikelompokkan ke dalam K kelompok yang berbeda. Setiap titik data termasuk dalam kelompok yang memiliki pusat terdekat dengan titik tersebut.

Clustering k-means bertujuan untuk meminimalkan jumlah kuadrat dalam-kelompok (within-cluster sum of squares), juga dikenal sebagai inersia atau squared error. Algoritma ini secara iteratif mencoba mencari pusat kelompok optimal yang meminimalkan fungsi objektif tersebut.

Perlu diingat bahwa algoritma K-means dapat konvergen ke optimum lokal, yang berarti hasilnya dapat bergantung pada pemilihan pusat awal yang acak. Untuk mengatasi masalah ini, umumnya algoritma ini dijalankan beberapa kali dengan inisialisasi yang berbeda, kemudian solusi klastering dengan inersia terendah dipilih.

Clustering k-means memiliki berbagai aplikasi dalam analisis data, kompresi gambar, segmentasi pelanggan, dan bidang lain di mana pengelompokan titik-titik data yang mirip digunakan untuk memahami pola dan hubungan dalam data.

In [ ]:
song_cluster_pipeline = Pipeline([('scaler', StandardScaler()), 
                                  ('kmeans', KMeans( init='k-means++', max_iter=1000,
                                   verbose=False,))
                                 ], verbose=False)

# song_cluster_pipeline = Pipeline([('scaler', StandardScaler()), 
#                                   ('kmeans', KMeans(n_clusters=5, init='k-means++', n_init=50, max_iter=500, random_state=42, 
#                                    verbose=False,))
#                                  ], verbose=False)


X = data.select_dtypes(np.number)
number_cols = list(X.columns)
song_cluster_pipeline.fit(X)
song_cluster_labels = song_cluster_pipeline.predict(X)
data['cluster_label'] = song_cluster_labels

# Visualizing the Clusters with PCA
from sklearn.decomposition import PCA

pca_pipeline = Pipeline([('scaler', StandardScaler()), ('PCA', PCA(n_components=2))])
song_embedding = pca_pipeline.fit_transform(X)
projection = pd.DataFrame(columns=['x', 'y'], data=song_embedding)
projection['title'] = data['name']
projection['cluster'] = data['cluster_label']

fig = px.scatter(
    projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'title'])
fig.show()

Kita menggunakan library sklearn dengan function sklearn.cluster.KMeans
Function ini mempunyai beberapa parameter, antara lain 
- **n_clusters**, jumlah cluster yang dipilih. Jika tidak diisi maka cluster akan di generate secara otomatis, tergantung fitur datanya.
- **n_init or init**, berapa kali algoritma akan dijalankan menggunakan awalan (seed) centroid yang berbeda. nilai default dari n_init adalah 10, sedangkan untuk init adalah 1
- **max_iter**, jumlah iterasi dari algoritma k-means yang dijalankan setiap run
- **tol**, singkatan dari *tolerance*, adalah toleransi relatif untuk menyatakan konvergensi. nilai lebih besar berarti lebih banyak konvergensi.
- **verbose**, jika true code akan menghasilkan statistik detil
- **random_state**, Menentukan pembangkitan angka acak untuk inisialisasi pusat klaster. Gunakan bilangan bulat (int) untuk membuat randomness menjadi deterministik.
- **copy_x**, jika false, data asli akan diubah sedikit untuk mempercepat proses training. Jika true maka data tidak diubah.
- **algorithm**, menentukan algoritma k-means yang akan digunakan. default adalah 'lloyd'. Variasi 'elkan' lebih efisien untuk data clustering yang nilainya sangat jelas.

Parameter yang kita gunakan adalah **init=k-means++** untuk mempercepat kalkulasi konvergensi dan **max_iter=1000** untuk menambah akurasi clustering. 

## 4. Kesimpulan

Untuk project machine learning clustering ini, kita menggunakan dataset lagu yang diambil dari spotify. Ada 170653 buah lagu dan 18 fitur. Untuk proses data cleaning, kita cukup menghilangkan data duplikat lagu dan lagu yg tidak mempunyai nilai apapun.  

Untuk memperdalam pemahaman kita terhadap dataset, kita melakukan feature corellation. Hasilnya adalah lagu yang populer cenderung rilis di tahun yang lebih baru, mempunyai tingkat energi yang tinggi, tingkat akustik rendah, dan volume yang cukup keras.  

Kita melakukan clustering dataset menggunakan algoritma K-Means dengan titik cluster yang diatur secara otomatis. Adapun parameter yang kita masukkan ke algoritma yaitu **init=k-means++** untuk mempercepat kalkulasi konvergensi dan **max_iter=1000** untuk menambah akurasi clustering.

Hasilnya dataset terbagi menjadi 7 buah cluster, dengan masing masing cluster mempunyai centroid yang cukup terdefinisi, kecuali cluster 2,5 dan 7 (bagian kiri atas) yang sangat tercampur. Setelah proses inspeksi, cluster tersebut terdiri dari lagu lagu yang cukup modern dan bervariasi. 